In [ ]:
#1 准备样本  2 对样本进行HOG+SVM训练  3  预测
#1.1样本  pos正样本 包含所检测的目标  neg 负样本不包含所检测的目标  尺寸调整64*128
#1.2  如何获取样本 1 网络 2公司内部 3  自己收集
#一个好的样本远胜一个复杂的神经网络   积极学习 几千个 深度学习 几兆个
#1.2.1  网络公司  一张图 一块钱  贵
#1.2.2  自己爬取
# 1.2.3   公司内部的积累 红外图像
#1.2.4  自己收集  视频的方式  100*30
# 正（负）样本的特点 尽可能的多样化  环境多样（干扰多样）
#  820pos  1931neg   1;2  1;3
#  name  规律


In [ ]:
#训练 
#1 参数的设置  2 创建一个HOG  3  SVM   4 计算HOG 5 label标签  6 完成训练 7 完成预测  8 绘图
import cv2
import numpy as np
import matplotlib.pyplot as plt
PosNum = 820
NegNum = 1931
winSize = (64,128)
blockSize = (16,16)#105
blocksStride = (8,8)#步长
cellSize = (8,8)#不移动
nBin = (9)#3780

#2 hog创建
hog = cv2.HOGDescriptor(winSize,blockSize,blocksStride,cellSize,nBin)#创建hog
#3  svm创建
svm = cv2.ml.SVM_create()
# 4  计算hog
featureNum = int(((128-16)/8+1)*((64-16)/8+1)*4*9)#3780
featureArray = np.zeros(((PosNum+NegNum),featureNum),np.float32)#二维  行列
labelArray = np.zeros((PosNum+NegNum,1),np.int32)#维度为1
#  设置原因 上   svm  监督学习   需要样本标签  学习的是图片hog特征  
for i in range(0,PosNum):
    fileName = 'pos\\'+str(i+1)+'.jpg'#win下用双斜杠
    img = cv2.imread(fileName)#省略  彩色
    hist = hog.compute(img,(8,8))#文件  hog    3780维度
    for j in range(0,featureNum):
        featureArray[i,j] = hist[j]#   featureArray装载hog特征 【1，：】 hog1[2,:]hog2
    labelArray[i,0] = 1#正样本 标签是1

for i in range(0,NegNum):
    fileName = 'neg\\'+str(i+1)+'.jpg'
    img = cv2.imread(fileName)#省略  彩色
    hist = hog.compute(img,(8,8))
    for j in range(0,featureNum):
        featureArray[i+PosNum,j] = hist[j]
        labelArray[i+PosNum,0] = -1

#正样本 1  负样本-1
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
#训练
ret = svm.train(featureArray,cv2.ml.ROW_SAMPLE,labelArray)#ret返回值   svm来源
#检测   没有用svmpredtict
#resultaway  rho 如何计算（来源svm)
alpha = np.zeros((1),np.float32)
rho = svm.getDecisionFunction(0,alpha)#来源 get方法
print(rho)
print(alpha)
alphaArray = np.zeros((1,1),np.float32)
supportVArray = np.zeros((1,featureNum),np.float32)#支持向量机数组
resultArray = np.zeros((1,featureNum),np.float32)
alphaArray[0,0] = alpha#设置alphaArray内容 一行一列
resultArray = -1*alphaArray*supportVArray# 在需要时先计算  支持向量机
#构建detect
myDetect = np.zeros((3781),np.float32)
for i in range(0,3780):#受featureNum限制
    myDetect[i] = resultArray[0,i]

myDetect[3780] = rho[0]#最后维度来源
#构建hog
#rho svm  阈值判决 累加时候起作用   
myHog = cv2.HOGDescriptor()
myHog.setSVMDetector(myDetect)#返回一个Hog  传递属性
imageSrc = cv2.imread('Test2.jpg',1)
#（8，8） 滑动步长  32，32  窗体大小
#核心 myhog
objs = myHog.detectMultiScale(imageSrc,0,(8,8),(32,32),1.05,2)# 返回检测结果data 0  8*8 32*32 缩放 2
#  xy wh   三维信息  参数放在最后一维
x = int(objs[0][0][0])# objs 是三维类型
y = int(objs[0][0][1])
w = int(objs[0][0][2])
h = int(objs[0][0][3])
cv2.rectangle(imageSrc,(x,y),(x+w,y+h),(255,0,0),2)
cv2.imshow('dst',imageSrc)
cv2.waitKey(0)

(0.19288761757743814, array([[1.]]), array([[0]], dtype=int32))
[0.]
